# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [54]:
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, metrics
import tensorflow as tf
import matplotlib.pyplot as plt

In [55]:
houses = pd.read_csv("cal_housing.data", sep=",", names=['longitude', 'latitude', 'housingMedianAge', 'totalRooms', 'totalBedrooms', 'population', 'households','medianIncome','medianHouseValue'])

In [56]:
houses.drop(labels=['longitude', 'latitude'], inplace=True, axis=1)

In [57]:
houses.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [58]:
houses.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [59]:
train_set, test_set = model_selection.train_test_split(houses[:-1], test_size=0.3)

In [60]:
train_set.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
4617,26.0,2107.0,757.0,2660.0,740.0,2.3375,282300.0
3835,27.0,3440.0,739.0,1827.0,712.0,4.1250,245500.0
8190,36.0,1245.0,211.0,508.0,221.0,5.3441,480600.0
16767,35.0,2644.0,456.0,1465.0,430.0,4.9375,277000.0
7571,39.0,1565.0,364.0,1389.0,360.0,2.7443,113900.0


In [61]:
test_set.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
344,43.0,1036.0,233.0,652.0,213.0,2.0690,84600.0
13707,32.0,1677.0,354.0,1021.0,339.0,3.6853,90900.0
17489,28.0,1992.0,531.0,1622.0,509.0,2.7689,228200.0
9886,52.0,144.0,54.0,89.0,48.0,1.0096,162500.0
8077,19.0,2953.0,895.0,1914.0,855.0,3.5521,290000.0


In [62]:
X_train, y_train, X_test, y_test = train_set.iloc[:,:-1], train_set.iloc[:,-1], test_set.iloc[:,:-1], test_set.iloc[:,-1]

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [63]:
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)
X_train_prep = scaler.transform(X_train)
X_test_prep = scaler.transform(X_test)

In [64]:
X_train_df = pd.DataFrame(data=X_train_prep,columns = X_train.columns,index=X_train.index)
X_test_df = pd.DataFrame(data=X_test_prep, columns = X_test.columns,index=X_test.index)

In [65]:
X_test_df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
344,0.823529,0.026298,0.035853,0.018190,0.034704,0.108212
13707,0.607843,0.042601,0.054633,0.028532,0.055428,0.219680
17489,0.529412,0.050613,0.082105,0.045377,0.083388,0.156481
9886,1.000000,0.003612,0.008071,0.002410,0.007566,0.035151
8077,0.352941,0.075055,0.138600,0.053561,0.140296,0.210494


### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [66]:
feat_columns = [tf.feature_column.numeric_column(i, shape=[1]) for i in X_train_df.columns]
feat_columns

[_NumericColumn(key='housingMedianAge', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='totalRooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='totalBedrooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='population', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='households', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='medianIncome', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [67]:
y_train.head()

4617     282300.0
3835     245500.0
8190     480600.0
16767    277000.0
7571     113900.0
Name: medianHouseValue, dtype: float64

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

### Input function V1

In [68]:
input_funct1 = tf.estimator.inputs.pandas_input_fn(x=X_train_df,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [69]:
estimator_dnn1 = tf.estimator.DNNRegressor(feature_columns = feat_columns, hidden_units=[6,10,15,10,6])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\F33C~1\\AppData\\Local\\Temp\\tmpb3wbkwb0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000AB9F7EB7F0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [70]:
estimator_dnn1.train(input_fn=input_funct1, steps=30000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\F33C~1\AppData\Local\Temp\tmpb3wbkwb0\model.ckpt.
INFO:tensorflow:loss = 699571770000.0, step = 1
INFO:tensorflow:global_step/sec: 377.101
INFO:tensorflow:loss = 482824100000.0, step = 101 (0.271 sec)
INFO:tensorflow:global_step/sec: 561.426
INFO:tensorflow:loss = 80575170000.0, step = 201 (0.175 sec)
INFO:tensorflow:global_step/sec: 460.522
INFO:tensorflow:loss = 171629050000.0, step = 301 (0.219 sec)
INFO:tensorflow:global_step/sec: 534.404
INFO:tensorflow:loss = 45185413000.0, step = 401 (0.185 sec)
INFO:tensorflow:global_step/sec: 471.382
INFO:tensorflow:loss = 42219856000.0, step = 501 (0.213 sec)
INFO:tensorflow:global_step/sec: 471.384
INFO:tensorflow:loss = 71312350000.0, step = 601 (0.214 s

INFO:tensorflow:global_step/sec: 328.727
INFO:tensorflow:loss = 61713790000.0, step = 7701 (0.303 sec)
INFO:tensorflow:global_step/sec: 337.615
INFO:tensorflow:loss = 72604200000.0, step = 7801 (0.296 sec)
INFO:tensorflow:global_step/sec: 489.868
INFO:tensorflow:loss = 115418170000.0, step = 7901 (0.204 sec)
INFO:tensorflow:global_step/sec: 446.131
INFO:tensorflow:loss = 29854690000.0, step = 8001 (0.223 sec)
INFO:tensorflow:global_step/sec: 401.338
INFO:tensorflow:loss = 34651040000.0, step = 8101 (0.252 sec)
INFO:tensorflow:global_step/sec: 398.139
INFO:tensorflow:loss = 50757470000.0, step = 8201 (0.253 sec)
INFO:tensorflow:global_step/sec: 342.226
INFO:tensorflow:loss = 43742840000.0, step = 8301 (0.290 sec)
INFO:tensorflow:global_step/sec: 297.427
INFO:tensorflow:loss = 123960540000.0, step = 8401 (0.334 sec)
INFO:tensorflow:global_step/sec: 351.881
INFO:tensorflow:loss = 19731098000.0, step = 8501 (0.288 sec)
INFO:tensorflow:global_step/sec: 469.168
INFO:tensorflow:loss = 1227938

INFO:tensorflow:global_step/sec: 458.409
INFO:tensorflow:loss = 72169450000.0, step = 15601 (0.218 sec)
INFO:tensorflow:global_step/sec: 487.481
INFO:tensorflow:loss = 32949981000.0, step = 15701 (0.201 sec)
INFO:tensorflow:global_step/sec: 448.13
INFO:tensorflow:loss = 37665264000.0, step = 15801 (0.226 sec)
INFO:tensorflow:global_step/sec: 499.666
INFO:tensorflow:loss = 61261820000.0, step = 15901 (0.200 sec)
INFO:tensorflow:global_step/sec: 494.718
INFO:tensorflow:loss = 31988328000.0, step = 16001 (0.200 sec)
INFO:tensorflow:global_step/sec: 489.869
INFO:tensorflow:loss = 176292560000.0, step = 16101 (0.209 sec)
INFO:tensorflow:global_step/sec: 460.523
INFO:tensorflow:loss = 58440528000.0, step = 16201 (0.215 sec)
INFO:tensorflow:global_step/sec: 471.382
INFO:tensorflow:loss = 39414280000.0, step = 16301 (0.210 sec)
INFO:tensorflow:global_step/sec: 502.179
INFO:tensorflow:loss = 67388457000.0, step = 16401 (0.201 sec)
INFO:tensorflow:global_step/sec: 473.611
INFO:tensorflow:loss = 

INFO:tensorflow:global_step/sec: 448.131
INFO:tensorflow:loss = 47030990000.0, step = 23501 (0.223 sec)
INFO:tensorflow:global_step/sec: 492.282
INFO:tensorflow:loss = 26589290000.0, step = 23601 (0.200 sec)
INFO:tensorflow:global_step/sec: 485.113
INFO:tensorflow:loss = 64679260000.0, step = 23701 (0.207 sec)
INFO:tensorflow:global_step/sec: 502.177
INFO:tensorflow:loss = 19794014000.0, step = 23801 (0.202 sec)
INFO:tensorflow:global_step/sec: 499.665
INFO:tensorflow:loss = 162712980000.0, step = 23901 (0.197 sec)
INFO:tensorflow:global_step/sec: 504.715
INFO:tensorflow:loss = 47465610000.0, step = 24001 (0.198 sec)
INFO:tensorflow:global_step/sec: 502.177
INFO:tensorflow:loss = 45213670000.0, step = 24101 (0.199 sec)
INFO:tensorflow:global_step/sec: 485.113
INFO:tensorflow:loss = 76792180000.0, step = 24201 (0.212 sec)
INFO:tensorflow:global_step/sec: 499.666
INFO:tensorflow:loss = 28610804000.0, step = 24301 (0.194 sec)
INFO:tensorflow:global_step/sec: 480.447
INFO:tensorflow:loss =

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [71]:
predict_input_func1 = tf.estimator.inputs.pandas_input_fn(x=X_test_df, batch_size=10,num_epochs=1,shuffle=False)

In [72]:
pred_gen = estimator_dnn1.predict(input_fn=predict_input_func1)
predictions=list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\F33C~1\AppData\Local\Temp\tmpb3wbkwb0\model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [73]:
final_predicts=[i['predictions'] for i in predictions]

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [74]:
from sklearn.metrics import mean_squared_error

rmse1 = mean_squared_error(y_test,final_predicts)**0.5
rmse1

79459.60445871446

### Input function V2

In [75]:
input_funct2 = tf.estimator.inputs.pandas_input_fn(x=X_train_df,y=y_train, batch_size=100, num_epochs=200, shuffle=True)
estimator_dnn2 = tf.estimator.DNNRegressor(feature_columns = feat_columns, hidden_units=[3,12,18,24,18,12,3])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\F33C~1\\AppData\\Local\\Temp\\tmpe4zi6nl3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000AB9F684F98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [76]:
estimator_dnn2.train(input_fn=input_funct2, steps=20000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\F33C~1\AppData\Local\Temp\tmpe4zi6nl3\model.ckpt.
INFO:tensorflow:loss = 5993012400000.0, step = 1
INFO:tensorflow:global_step/sec: 301.39
INFO:tensorflow:loss = 1319823000000.0, step = 101 (0.336 sec)
INFO:tensorflow:global_step/sec: 367.401
INFO:tensorflow:loss = 757424200000.0, step = 201 (0.271 sec)
INFO:tensorflow:global_step/sec: 378.536
INFO:tensorflow:loss = 896438040000.0, step = 301 (0.264 sec)
INFO:tensorflow:global_step/sec: 378.535
INFO:tensorflow:loss = 617250800000.0, step = 401 (0.264 sec)
INFO:tensorflow:global_step/sec: 382.887
INFO:tensorflow:loss = 951586460000.0, step = 501 (0.263 sec)
INFO:tensorflow:global_step/sec: 367.401
INFO:tensorflow:loss = 610335060000.0, step = 601 (0.

INFO:tensorflow:loss = 598616700000.0, step = 7601 (0.267 sec)
INFO:tensorflow:global_step/sec: 385.842
INFO:tensorflow:loss = 623948300000.0, step = 7701 (0.259 sec)
INFO:tensorflow:global_step/sec: 382.887
INFO:tensorflow:loss = 453501100000.0, step = 7801 (0.261 sec)
INFO:tensorflow:global_step/sec: 382.885
INFO:tensorflow:loss = 409283300000.0, step = 7901 (0.261 sec)
INFO:tensorflow:global_step/sec: 388.846
INFO:tensorflow:loss = 627676740000.0, step = 8001 (0.257 sec)
INFO:tensorflow:global_step/sec: 393.438
INFO:tensorflow:loss = 452231140000.0, step = 8101 (0.253 sec)
INFO:tensorflow:global_step/sec: 394.991
INFO:tensorflow:loss = 560799350000.0, step = 8201 (0.255 sec)
INFO:tensorflow:global_step/sec: 374.283
INFO:tensorflow:loss = 413384180000.0, step = 8301 (0.268 sec)
INFO:tensorflow:global_step/sec: 382.886
INFO:tensorflow:loss = 792142400000.0, step = 8401 (0.259 sec)
INFO:tensorflow:global_step/sec: 371.498
INFO:tensorflow:loss = 413466170000.0, step = 8501 (0.269 sec)
I

INFO:tensorflow:global_step/sec: 371.5
INFO:tensorflow:loss = 823879800000.0, step = 15501 (0.268 sec)
INFO:tensorflow:global_step/sec: 390.365
INFO:tensorflow:loss = 493476640000.0, step = 15601 (0.255 sec)
INFO:tensorflow:global_step/sec: 385.842
INFO:tensorflow:loss = 413792470000.0, step = 15701 (0.260 sec)
INFO:tensorflow:global_step/sec: 378.535
INFO:tensorflow:loss = 550422800000.0, step = 15801 (0.269 sec)
INFO:tensorflow:global_step/sec: 384.358
INFO:tensorflow:loss = 618709800000.0, step = 15901 (0.255 sec)
INFO:tensorflow:global_step/sec: 387.338
INFO:tensorflow:loss = 496646200000.0, step = 16001 (0.258 sec)
INFO:tensorflow:global_step/sec: 391.896
INFO:tensorflow:loss = 760157960000.0, step = 16101 (0.255 sec)
INFO:tensorflow:global_step/sec: 345.789
INFO:tensorflow:loss = 540983660000.0, step = 16201 (0.290 sec)
INFO:tensorflow:global_step/sec: 384.359
INFO:tensorflow:loss = 653965260000.0, step = 16301 (0.259 sec)
INFO:tensorflow:global_step/sec: 344.597
INFO:tensorflow:

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [77]:
predict_input_func2 = tf.estimator.inputs.pandas_input_fn(x=X_test_df, batch_size=10,num_epochs=1,shuffle=False)

In [78]:
pred_gen = estimator_dnn2.predict(input_fn=predict_input_func2)
predictions=list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\F33C~1\AppData\Local\Temp\tmpe4zi6nl3\model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [79]:
from sklearn.metrics import mean_squared_error

final_predicts=[i['predictions'] for i in predictions]

rmse2 = mean_squared_error(y_test,final_predicts)**0.5

In [80]:
rmse2

75218.31134179124

### Input function V3

In [81]:
input_funct3 = tf.estimator.inputs.pandas_input_fn(x=X_train_df,y=y_train, batch_size=50, num_epochs=500, shuffle=False)
estimator_dnn3 = tf.estimator.DNNRegressor(feature_columns = feat_columns, hidden_units=[32,64,32])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\F33C~1\\AppData\\Local\\Temp\\tmpjanblv_e', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000AB922CB080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [82]:
estimator_dnn3.train(input_fn=input_funct3, steps=20000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\F33C~1\AppData\Local\Temp\tmpjanblv_e\model.ckpt.
INFO:tensorflow:loss = 3362217800000.0, step = 1
INFO:tensorflow:global_step/sec: 404.587
INFO:tensorflow:loss = 1005918900000.0, step = 101 (0.250 sec)
INFO:tensorflow:global_step/sec: 434.491
INFO:tensorflow:loss = 575308400000.0, step = 201 (0.231 sec)
INFO:tensorflow:global_step/sec: 440.237
INFO:tensorflow:loss = 443731840000.0, step = 301 (0.228 sec)
INFO:tensorflow:global_step/sec: 421.659
INFO:tensorflow:loss = 626175840000.0, step = 401 (0.236 sec)
INFO:tensorflow:global_step/sec: 428.898
INFO:tensorflow:loss = 587509600000.0, step = 501 (0.232 sec)
INFO:tensorflow:global_step/sec: 442.183
INFO:tensorflow:loss = 447248400000.0, step = 601 (0

INFO:tensorflow:loss = 345013980000.0, step = 7601 (0.234 sec)
INFO:tensorflow:global_step/sec: 425.247
INFO:tensorflow:loss = 172398400000.0, step = 7701 (0.235 sec)
INFO:tensorflow:global_step/sec: 450.152
INFO:tensorflow:loss = 183801070000.0, step = 7801 (0.222 sec)
INFO:tensorflow:global_step/sec: 452.186
INFO:tensorflow:loss = 306496440000.0, step = 7901 (0.222 sec)
INFO:tensorflow:global_step/sec: 452.187
INFO:tensorflow:loss = 336903570000.0, step = 8001 (0.221 sec)
INFO:tensorflow:global_step/sec: 446.13
INFO:tensorflow:loss = 248281420000.0, step = 8101 (0.223 sec)
INFO:tensorflow:global_step/sec: 448.13
INFO:tensorflow:loss = 225005490000.0, step = 8201 (0.225 sec)
INFO:tensorflow:global_step/sec: 452.187
INFO:tensorflow:loss = 277521560000.0, step = 8301 (0.219 sec)
INFO:tensorflow:global_step/sec: 419.888
INFO:tensorflow:loss = 220136780000.0, step = 8401 (0.238 sec)
INFO:tensorflow:global_step/sec: 450.149
INFO:tensorflow:loss = 247450350000.0, step = 8501 (0.225 sec)
INF

INFO:tensorflow:global_step/sec: 436.392
INFO:tensorflow:loss = 239148040000.0, step = 15501 (0.231 sec)
INFO:tensorflow:global_step/sec: 448.132
INFO:tensorflow:loss = 374591600000.0, step = 15601 (0.222 sec)
INFO:tensorflow:global_step/sec: 448.128
INFO:tensorflow:loss = 243628980000.0, step = 15701 (0.222 sec)
INFO:tensorflow:global_step/sec: 444.147
INFO:tensorflow:loss = 381892170000.0, step = 15801 (0.226 sec)
INFO:tensorflow:global_step/sec: 430.75
INFO:tensorflow:loss = 313935170000.0, step = 15901 (0.232 sec)
INFO:tensorflow:global_step/sec: 452.187
INFO:tensorflow:loss = 353370240000.0, step = 16001 (0.222 sec)
INFO:tensorflow:global_step/sec: 440.235
INFO:tensorflow:loss = 466418430000.0, step = 16101 (0.225 sec)
INFO:tensorflow:global_step/sec: 454.238
INFO:tensorflow:loss = 234288960000.0, step = 16201 (0.221 sec)
INFO:tensorflow:global_step/sec: 452.191
INFO:tensorflow:loss = 176470070000.0, step = 16301 (0.221 sec)
INFO:tensorflow:global_step/sec: 454.241
INFO:tensorflow

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [83]:
predict_input_func3 = tf.estimator.inputs.pandas_input_fn(x=X_test_df, batch_size=10,num_epochs=1,shuffle=False)

In [84]:
pred_gen = estimator_dnn3.predict(input_fn=predict_input_func3)
predictions=list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\F33C~1\AppData\Local\Temp\tmpjanblv_e\model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [85]:
final_predicts=[i['predictions'] for i in predictions]

rmse3 = mean_squared_error(y_test,final_predicts)**0.5

In [86]:
rmse1, rmse2, rmse3

(79459.60445871446, 75218.31134179124, 78518.5195844361)

# Great Job!